In [ ]:
#| default_exp markdown.obsidian.personal.note_processing

In [ ]:
#| export
from deprecated import deprecated
from os import PathLike
from pathlib import Path, PureWindowsPath, WindowsPath
import re
from typing import Optional, Union

from trouver.markdown.markdown.file import MarkdownFile, MarkdownLineEnum, replace_embedded_links_with_text
from trouver.markdown.obsidian.footnotes import remove_footnote_mentions_in_markdown_text
from trouver.markdown.obsidian.links import EMBEDDED_PATTERN, remove_links_from_text
from trouver.markdown.obsidian.vault import VaultNote

# markdown.obisidian.personal.note_processing
> Process notes to extract basic information about them 

For the `machine_learning` module, 

In [ ]:
# TODO: add description

In [ ]:
#| export
def remove_double_asterisks_in_markdown_file(
        markdown_file: MarkdownFile 
        ) -> None: 
    # TODO Don't remove in math mode.
    """Remove double asterisks in MarkdownFile object.
    
    I chose not to implement this method in the `MarkdownFile` class because it
    seems to specific. 
    
    **Parameters**
    - markdown_file - MarkdownFile
    """
    for part in markdown_file.parts:
        part['line'] = part['line'].replace('**', '')

In [ ]:
#| export
def standard_information_note_processor_v2(
        markdown_file: Union[MarkdownFile, str],
        vault: PathLike,
        remove_frontmatter_meta: bool = True,
        remove_see_also_section: bool = True,
        remove_meta_section: bool = True,
        remove_references_section: bool = True,
        remove_double_asterisks: bool = True,
        remove_links: bool = True,
        remove_in_line_tags: bool = True,
        remove_footnotes_to_embedded: bool = True,
        remove_headers: bool = True,
        remove_citation_footnotes: bool = True,
        replace_embedded_links_with_content: bool = True,
        merge_in_line_latex: bool = True,
        merge_in_line_latex_into_text: Optional[str] = None,
        no_double_blank_lines: bool = True) -> MarkdownFile:
    """Processes a str of a standard information note.
        
    The note processing is for
    
    - note type categorization
    - definition finding
    
    TODO: implement remove_citation_footnote properly.
    
    **Parameters**
    - markdown_file - MarkdownFile or str
    - vault - PathLike
    - remove_frontmatter_meta - bool
        - If `True`, removes the frontmatter meta. Defaults to `True`
    - remove_see_also_section - bool
        - If `True`, removes the `# See also` section. Defaults to `True`.
    - remove_meta_section - bool
        - If `True`, remove the `# Meta` section. Defaults to `True`.
    - remove_references_section - bool
        - If `True`, removes the `## References` section. Defaults to `True`.
    - remove_double_asterisks - bool
        - If `True`, removes double asterisks. Defaults to `True`.
    - remove_links - bool
        - If `True`, removes nonembedded links and replaces them with their
        display text. Defaults to `True`.
    - remove_in_line_tags - bool
        - If `True`, removes in-line tags (the lines that start with a tag).
        Defaults to `True`.
    - remove_footnotes_to_embedded - bool
        - If `True`, removes footnotes to embedded notes. Defaults to `True`.
    = remove_headers - bool
        - If `True`, removes headers. Defaults to `True`.
    - remove_citation_footnote - bool 
        - If `True`, removes the citation footnote. Defaults to `True`.
    - replace_embedded_links_with_content - bool
        - If `True`, replaces embedded links with their content.
        Defaults to `True`.
    - merge_in_line_latex - bool
        - If `True`, merge each group of in-line latex lines into single lines.
        Defaults to `True`.
    - merge_in_line_latex_into_text - str or `None`
        - If not `None`, merge each group of in-line latex lines into
        single lines and merge those groups into the text that precedes
        them with the specified str. Defaults to `None`.
        The blank character ` ` and the new-line character `\n` are recommended
        as arguments.
    - no_double_blank_lines - bool
        - If `True`, removes escape characters `'\n'` to make it so that
        there are no double blank lines. Defaults to `True`.
    """
    if isinstance(markdown_file, str):
        markdown_file = MarkdownFile.from_string(markdown_file)
    if remove_frontmatter_meta:
        markdown_file.remove_metadata()
    # if not remove_citation_footnote:
    #     return
    if remove_footnotes_to_embedded:
        markdown_file.remove_footnotes_to_embedded_links()
    if replace_embedded_links_with_content:
        markdown_file.replace_embedded_links_with_text(vault)
    if remove_see_also_section:
        markdown_file.remove_section('See Also')
    if remove_meta_section:
        markdown_file.remove_section('Meta')
    if remove_references_section:
        markdown_file.remove_section('References')
    if remove_double_asterisks:
        remove_double_asterisks_in_markdown_file(markdown_file)
    if remove_links: 
        markdown_file.replace_links_with_display_text()
    if remove_in_line_tags:
        markdown_file.remove_in_line_tags()
    if remove_headers:
        markdown_file.remove_headers()
    if merge_in_line_latex:
        markdown_file.merge_in_line_latex()
    if merge_in_line_latex_into_text:
        markdown_file.merge_in_line_latex_into_preceding_text(
            separator=merge_in_line_latex_into_text)
    if no_double_blank_lines:
        markdown_file.remove_double_blank_lines()
    # print(markdown_file)
    return markdown_file
